# [M_02] PROTOKÓŁ: AGENT STRATEGII I COPYWRITINGU

**PROJEKT:** OMNI-OPERATOR-V1  
**SILNIK:** GEMINI 3 FLASH  
**STATUS:** GENEROWANIE_NARRACJI

Ten moduł implementuje logikę Agenta Strategii. Jego zadaniem jest transformacja ustrukturyzowanych danych z analizy wideo (M_01) w perswazyjne treści dopasowane do specyfiki różnych platform social media.

**Cele operacyjne:**
1. Generowanie unikalnych postów dla TikTok, YouTube Shorts oraz LinkedIn.
2. Automatyczny dobór viralowych hashtagów.
3. Pełny tracing operacji w **Langfuse v2**, zapewniający kontrolę nad kosztami i logiką "myślenia" agenta.

In [1]:
import os
import sys
from typing import List
from pydantic import BaseModel, Field
from dotenv import load_dotenv

# Importy dla stabilnej wersji Langfuse v2 i PydanticAI
from langfuse.decorators import observe, langfuse_context
from pydantic_ai import Agent
from pydantic_ai.models.google import GoogleModel

# 1. KOREKTA ŚCIEŻKI ROBOCZEJ
if os.getcwd().endswith("notebooks"):
    os.chdir("..")

# Dodanie src do path
sys.path.append(os.path.join(os.getcwd(), "src"))

from src.core.config import settings
load_dotenv(".env")

# 2. UZBROJENIE KONFIGURACJI (Standard Gemini 3 Flash)
os.environ["GOOGLE_API_KEY"] = settings.gemini_api_key
os.environ["LANGFUSE_PUBLIC_KEY"] = settings.langfuse_public_key
os.environ["LANGFUSE_SECRET_KEY"] = settings.langfuse_secret_key
os.environ["LANGFUSE_HOST"] = "http://localhost:3000"

# 3. INICJALIZACJA SILNIKA
model = GoogleModel('gemini-3-flash-preview')

print(f"LOG: Jednostka uzbrojona. Katalog ROOT: {os.getcwd()}")
print(f"LOG: Monitoring Langfuse v2 aktywny.")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


LOG: Jednostka uzbrojona. Katalog ROOT: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1
LOG: Monitoring Langfuse v2 aktywny.


## 1. Definicja Kontraktu Copywritingu (Schema)

Budujemy modele danych, które wymuszają na Gemini 3 Flash dostarczenie kompletu postów dla każdego wyciętego klipu.

In [2]:
class PlatformPost(BaseModel):
    """Pojedyncza treść zoptymalizowana pod konkretny algorytm."""
    platform: str = Field(description="TikTok, YouTube lub LinkedIn")
    content: str = Field(description="Tekst posta z hookiem i CTA")
    hashtags: List[str] = Field(description="Lista celowanych hashtagów")

class ClipStrategy(BaseModel):
    """Strategia marketingowa dla pojedynczego fragmentu wideo."""
    clip_index: int = Field(description="Numer porządkowy klipu z analizy")
    duration_seconds: int = Field(description="Długość klipu w sekundach") # <--- NOWOŚĆ
    posts: List[PlatformPost] = Field(description="Warianty treści na platformy")

class CampaignBrief(BaseModel):
    """Kompletny brief operacyjny kampanii."""
    overall_strategy: str = Field(description="Główna idea i ton kampanii")
    clip_strategies: List[ClipStrategy] = Field(description="Szczegółowe posty dla każdego klipu")

print("LOG: Kontrakt danych CampaignBrief zainicjalizowany.")

LOG: Kontrakt danych CampaignBrief zainicjalizowany.


## 2. Inicjalizacja Agenta Strategii (PydanticAI)

Inicjujemy agenta, definiując mu rolę Szefa Strategii. Wykorzystujemy `output_type`, aby zapewnić 100% kompatybilności z naszym schematem JSON.

In [3]:
copywriter_agent = Agent(
    model=model,
    output_type=CampaignBrief,
    system_prompt=(
        "Jesteś Szefem Strategii w KUŹNI OPERATORÓW. Twoim celem jest maksymalizacja zasięgów wideo. "
        "Na podstawie technicznej analizy materiału przygotuj porywające treści. "
        "Dopasuj dynamikę i długość copy do czasu trwania klipu (duration_seconds): "
        "- Dla klipów krótki (<30s): Stosuj ekstremalnie silne hooki i krótkie, prowokujące copy. "
        "- Dla klipów dłuższych (>30s): Dodaj jeden konkretny merytoryczny wniosek przed CTA. "
        "1. TikTok: Agresywny hook, krótka forma, dużo energii. "
        "2. YouTube: Edukacja, SEO i jasna wartość. "
        "3. LinkedIn: Budowanie autorytetu, merytoryczne wnioski, storytelling. "
        "Pisz wyłącznie po polsku."
    )
)
print("LOG: Agent Copywriter (Gemini 3 Flash) gotowy do akcji.")

LOG: Agent Copywriter (Gemini 3 Flash) gotowy do akcji.


## 3. Egzekucja Kampanii (Observability Mode)

Uruchamiamy proces generowania treści. Każde wywołanie jest automatycznie śledzone w panelu Langfuse dzięki dekoratorowi `@observe`.

In [4]:
# Dane wejściowe udające wynik z modułu M_01
mock_analysis = {
    "topic": "Suwerenność AI i własny stack technologiczny",
    "clips": [
        {"index": 1, "duration_seconds": 20, "hook": "Eliminacja SaaS"},
        {"index": 2, "duration_seconds": 55, "hook": "Prywatna baza Qdrant"}
    ]
}

@observe(name="Mission_Generate_Copy")
async def run_copywriting_mission():
    try:
        print("LOG: Agent analizuje dane i generuje treści...")
        
        # Wywołanie agenta (Wersja 1.39.0 używa .output)
        result = await copywriter_agent.run(
            f"Przygotuj kampanię dla tych klipów: {mock_analysis}"
        )
        
        brief = result.output
        
        print("\n" + "="*50)
        print(f"🚀 STRATEGIA: {brief.overall_strategy}")
        print("="*50)
        
        for strategy in brief.clip_strategies:
            print(f"\n🎬 KLIP #{strategy.clip_index}:")
            for post in strategy.posts:
                print(f"   [{post.platform.upper()}]")
                print(f"   📝 {post.content}")
                print(f"   #️⃣ {', '.join(post.hashtags)}\n")
        
        # Synchronizacja z Langfuse v2
        langfuse_context.flush()
        print("LOG: Dane wysłane do Langfuse.")
        
        return brief

    except Exception as e:
        print(f"❌ BŁĄD OPERACYJNY: {str(e)}")

# START
final_brief = await run_copywriting_mission()

LOG: Agent analizuje dane i generuje treści...

🚀 STRATEGIA: Kampania 'Manifest Suwerenności' – pozycjonujemy markę jako lidera buntu przeciwko nadmiernemu uzależnieniu od SaaS. Ton jest ekspercki, stanowczy i technicznie bezkompromisowy. Podkreślamy, że prawdziwa potęga AI leży w posiadaniu infrastruktury na własność.

🎬 KLIP #1:
   [TIKTOK]
   📝 SaaS Cię ogranicza i drenuje Twój budżet. 💸 Czas na brutalną prawdę: jeśli nie masz własnego stacku, nie masz kontroli nad swoim biznesem. Eliminujemy pośredników i budujemy suwerenność. Zobacz jak! 🛠️ #AI #Sovereignty #TechFreedom
   #️⃣ #automatyzacja, #biznes, #ai, #programowanie, #selfhosted

   [YOUTUBE]
   📝 Koniec z uzależnieniem od zewnętrznych dostawców. Omawiamy proces eliminacji SaaS na rzecz własnych rozwiązań AI. Dowiedz się, dlaczego suwerenność technologiczna to trend 2024 roku. 🚀
   #️⃣ #sztucznainteligencja, #technologia, #saas, #wdrozeniaai

   [LINKEDIN]
   📝 W dobie rosnących cen subskrypcji i obaw o prywatność danych, 'Sa

## STATUS: MODUŁ 02 ZAKOŃCZONY

System posiada teraz "mózg" marketingowy. Potrafi transformować surowe dane w perswazyjną komunikację.

**Następne działania:**
1. Zapisz notatnik i sprawdź logi na `localhost:3000`.
2. Przejdź do **M_03: Automatyczna Fabryka Wideo**, gdzie fizycznie wytniemy te klipy.